In [1]:
# Required packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras import optimizers

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
# load data
dataset = np.loadtxt("data_tp1", delimiter=",")
X_train = dataset[:, 1:].reshape(dataset.shape[0],1,28, 28).astype( 'float32' )
y_train = dataset[:,0]
X_test = dataset[:,1:].reshape(dataset.shape[0],1, 28, 28).astype( 'float32' )
y_test = dataset[:,0]

In [4]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = 784
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [5]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

In [6]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
# define model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, input_dim=25, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Set the gradient
    sgd = optimizers.SGD(lr=.5)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [8]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Training Error: %.2f%%" % (100-scores[1]*100))

Train on 5000 samples, validate on 5000 samples
Epoch 1/10
 - 1s - loss: 7.4649 - acc: 0.2186 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 2/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 3/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 4/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 5/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 6/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 7/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 8/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 9/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Epoch 10/10
 - 1s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.4096 - val_acc: 0.1060
Training Error: 89.40%
